# 선형회귀

보험료 예측하기

In [ ]:
#필요한 라이브러리 불러오기
import pandas as pd

In [ ]:
# https://www.kaggle.com/
# 1. 직접적으로 파일 경로
# 2. 외부의 링크를 전달 받아서
# https://raw.githubusercontent.com/BigDeepData/2312_as/main/data/insurance.csv
df = pd.read_csv('https://raw.githubusercontent.com/BigDeepData/2312_as/main/data/insurance.csv')
df

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95


In [ ]:
# df을 불러오면 4개를 해봐야합니다
# df.head, df.tail, df.info, df.describe
df.head()
# age : 나이, sex: 성별, bmi, children: 자녀수, region: 지역, expenses : 보험료
# 보험료 -> expenses : 종속변수, 목표변수(타겟변수)

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe() # 연속형 변수에 대한 통계치

In [ ]:
# 소수점 2자리 표시
pd.options.display.float_format = "{:.2f}".format

In [ ]:
df.describe()

In [ ]:
# df.describe(include="object")
df.describe(include="O")

#개인학습 문제

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# 1.연령별
sns.histplot(data=df, x='age')

In [ ]:
# 2. 보험료
sns.histplot(data=df, x='expenses')

In [ ]:
# 3.남녀비율
sns.barplot(data=df, x='sex', y='expenses')

## 데이터 전처리
* 데이터를 모델링할 수 있게 다듬는 과정
1. 하지 않으면 모델링 자체가 안되는 전처리
1. 모델의 성능을 높이기 위한 전처리

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
# 모델을 통해서 학습을 시켜주려면 X(독립변수들)에 있는 값들이 모두 숫자형태여야함

# 독립변수들(X) => 2개 이상의 열을 가진 df
X = df.drop('expenses', axis=1)
# 종속변수(y) => 1개
y = df['expenses']
try:
    model.fit(X, y) # 훈련
except Exception as e:
    print(type(e))
    print(e)
    # object로 표현되어 있는 범주형 변수들이 버그를 일으킴

#범주형 데이터 처리

In [ ]:
df.info()

In [ ]:
df.smoker

In [ ]:
# 고유값, 빈도
df.smoker.value_counts() # df.uninque(), df.nunique()

In [ ]:
# yes : 1, no : 0 # Bool T/F (1/0)
(df.smoker == 'yes') + 0
(df.smoker == 'yes') * 1

In [ ]:
# equal : 동일 여부를 연산
# mul : multply
df.smoker.eq('yes').mul(1) # (df.smoker == 'yes') * 1

In [ ]:
df.smoker = df.smoker.eq('yes').mul(1)
df.info()

### 범주형 데이터 처리 / 더미 변수 (원-핫 인코딩)
* sex : `is_male` ? / `is_female`?
* region : `ne`, `nw`, `se`, `sw` (0, 1, 1...)

In [ ]:
# pd.get_dummies(df, columns=[내가 변환시키고 싶은 칼럼 이름들])
pd.get_dummies(df, columns=['sex', 'region']) # n개의 고윳값 -> n-1열
# 열 -> 계산을 많이해줘야함

In [ ]:
# 필요없는 고윳값에 따른 열 1개씩을 삭제
pd.get_dummies(df, columns=['sex', 'region'], drop_first=True) # n개의 고윳값 -> n-1열

In [ ]:
df2 = df.copy()
df = pd.get_dummies(df, columns=['sex', 'region'], drop_first=True) # n개의 고윳값 -> n-1열
df

### 훈련셋 / 시험셋 나누기
* 데이터를 넣어서 모델을 학습시킨 다음에 해당 모델을 가지고 새로운 값을 넣어서 결과를 받아내는 프로그램
* 기존에 학습에 쓰인 데이터 X / 새로운 데이터를 넣어서 검증을 해줘야함
* 100개의 데이터가 있다 -> `70~80`개는 학습에 쓰고, `20~30`개 정도는 검증을 위한 (시험을 위한) 데이터 셋으로 남겨놓음
---
모델링 -> 데이터 분할 작업
1. 종속변수(y)와 독립변수(X)를 분리
    * 독립변수 -> (영향) -> 종속변수
2. 학습셋/훈련셋(train set)과 시험셋(test set) 분리
    * 학습의 결과에 대한 신뢰성, 정확성을 검증

||독립변수|종속변수|
|-|-|-|
|학습셋|X_train|y_train|
|시험셋|X_test|y_test|


### 독립변수와 종속변수
* **독립변수**
    * 예측에 사용되는 재료와 같은 변수들
    * 피쳐 (Feature), 피쳐 변수 (Feature Variable)
* **종속변수**
    * 예측의 대상이 되는 변수
    * 목표 변수, 타깃, 타깃 변수 (Target Variable)

> '지도 학습'에 속하는 모델들은 독립변수를 통해 종속변수를 예측하는 것이므로, 어떤 변수가 종속변수인지 명확하게 알려줘야함 -> 독립변수와 종속변수를 각각 별도의 데이터로 입력 받게 됨 (`model.fit(X:독립변수df, y:종속변수s)`)

### 학습(훈련)셋과 시험셋
> 학습셋과 시험셋을 구분하지 않고 예측 모델을 만들 경우 새로운 데이터에 대한 예측력을 검증할 수 X (기출문제를 학습시키고 다시 그 문제로 시험을 보는 것)
* 전체 데이터를 가지고 모델링(학습)을 하고, 또다시 전체 데이터에 대해서 예측값을 만들어서 종속변수와 비교 -> 특정 데이터에 대해 **과최적** 되어 있을 수 있음
* 시험셋을 통해 검증하지 않은 상태에서는 불확실성이 존재
* 일반적으로는 **학습셋:시험셋**을 각각 7:3, 8:2 정도로 나눔
* 만약 데이터가 너무 적다? -> 학습셋을 우선시 (데이터가 적다면 9:1도 가능...)

In [ ]:
df # y : expenses <- / X : 독립변수

In [ ]:
# 독립변수
X = df.drop('expenses', axis=1) # 독립변수들
# 종속변수
y = df['expenses'] # 목표변수/타깃변수/종속변수
X, y

In [ ]:
# 훈련셋 / 시험셋
from sklearn.model_selection import train_test_split # 훈련/시험 나눠주겠다

In [ ]:
# train_test_split(X: 독립변수들, y:종속변수, test_size=시험셋의 비율, random_state=랜덤값 기준)
# test_size : 전체 비율이 1이라고 했을 때, test set의 사이즈 (0.2~0.3)
# random_state : numpy.random.seed() -> 똑같은 값이 나와줄 수 있도록 기준을 잡아줌
# train_test_split(X, y) -> 4개 (1. X 훈련셋, 2. X 시험셋, 3. y 훈련셋 4. y 시험셋)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=15)

In [ ]:
len(df)

In [ ]:
# X(독립변수들) 훈련셋
X_train # 1338 * 0.8
# X(독립변수들) 시험셋
X_test # 1338 * 0.2
# y(종속변수) 훈련셋
y_train
# y(종속변수) 시험셋
y_test

## 모델 학습
> 모델링
* 머신러닝 알고리즘으로 모델을 학습시키는 과정 → 결과물 : 머신러닝 모델
* 모델링에 사용할 머신러닝 알고리즘을 선택
* 독립변수와 종속변수를 *fit()* 함수에 인수로 주어 학습

|알고리즘 선택| |모델 생성| |모델 학습| |모델을 사용한 예측|
|-|-|-|-|-|-|-|
|다양한 머신러닝<br>알고리즘 중 선택 후<br>라이브러리 임포트|→|모델링에 사용할<br>모델 생성|→|*fit()* 함수에 학습셋의<br>독립변수와 종속변수를<br>인수로 대입|→|train_test_split()<br>함수으로 생성된 평가셋의 <br>독립변수를 인수로 대입

In [ ]:
# 선형(linear) 회귀 모델
from sklearn.linear_model import LinearRegression

In [ ]:
# 선형회귀(LinearRegression) 클래스로 모델 생성
model = LinearRegression()

In [ ]:
# X 시험셋을 input해서 y_train의 정답을 가지도록 훈련
model.fit(X_train, y_train) # 모델 훈련 / 모델 학습

In [ ]:
# predict : 새로운 데이터를 넣어줘서 예측
pred = model.predict(X_test)
pred

#모델 평가

In [ ]:
# 테이블로 평가
comparison = pd.DataFrame({
    'actual': y_test, # 실제값
    'pred': pred
})
comparison

In [ ]:
# 그래프로 평가
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 산점도 그리기
plt.figure(figsize=(5, 5))
sns.scatterplot(x = 'actual', y= 'pred', data = comparison)
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))
sns.regplot(x = 'actual', y= 'pred', data = comparison)
plt.show()

### RMSE & R²(결정계수)
> 회귀 -> 연속형 변수를 예측할 때 쓰임 -> 값들의 나열 -> 예측값-실제값 사이의 차이 = 오차, 이 오차들을 기준으로 모델의 성능이 결정

|평가지표|설명|
|-|-|
|MAE (Mean Absolute Error, 평균 절대 오차)|- 실젯값과 예측값 사이의 오차에 절댓값을 씌운 뒤 이에 대한 평균을 계산<br>- 0에 가까울수록 좋음|
|MSE (Mean Squared Error, 평균 제곱 오차)|- 실젯값과 예측값 사이의 오차를  제곱한 뒤 이에 대한 평균을 계산<br>- 0에 가까울수록 좋음|
|RMSE (Root Mean Squared Error, 루트 평균 제곱 오차)<br>|- MSE에 루트를 씌운 값<br>- 0에 가까울수록 좋음<br>- 연속형 변수를 예측할 때 가장 일반적으로 사용되는 평가지표|
|R²|- 결정계수<br>- 독립변수가 종속변수를 얼마만큼 설명해 주는지 나타내는 지표, 즉 설명력<br>- 1에 가까울수록 좋음|

#mean_squared_error()

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
# mse -> 실제값, 예측값
mean_squared_error(y_test, pred) # MSE
# 모델 간 비교 (다른 알고리즘, 다른 독립변수 조합)

In [ ]:
# rmse
mean_squared_error(y_test, pred) ** 0.5 # 루트를 씌워주는 역할

In [ ]:
# rmse #2
mean_squared_error(y_test, pred, squared=False)

$R^2 = \frac{SSR}{SST}$<br>
* **SST** (Sum of Squares Total) : 총변동
* **SSR** (Sum of Squares Regression) : 설명 가능한 변동
* **SSE** : 설명이 안되는 변동
* 모델이 얼마나 값의 변동을 잘 설명하는가? = 예측을 잘하는가?
* https://m.blog.naver.com/pmw9440/221822183325

In [ ]:
# 학습시킨 독립변수, 종속변수
model.score(X_train, y_train) # R²

## 선형회귀 `Linear Regression`
* 독립변수와 종속변수 간에 선형 관계가 있음을 가정하여 최적의 선을 그려서 예측하는 방법
* 머신러닝에서는 손실(오차)함수 `Loss Function` 를 최소화하는 선을 찾아서 모델을 만들어냄
* 수식을 도출하기에 매우 쉽게 때문에 그 해석도 직관적
    * 회귀 직선의 수식에 관련된 값을 제공

In [ ]:
import numpy as np

# 독립변수별 계수 확인
model.coef_

In [ ]:
pd.Series(model.coef_, index = X.columns)

In [ ]:
model.intercept_

$261.94 × age + 339.54 × bmi + 607.58 × children + 23878.00 × smoker + 42.42× sex male + (-394.02) × region northwest + (-1073.15) × region southeast + (-973.02) × region southwest -12258.305052625143$

* 모델의 계수를 해석할 때 '부호'의 영향에 유의
* 부호와 상관없이 **계수의 절대값이 클수록 영향이 크다**라고 할 수 있고, 절대값이 0에 가까울 수록 영향력이 거의 없음
* 여러 계수를 서로 비교할 때, 단순히 절댓값이 더 크면 영향이 크다? -> 각 변수의 스케일(단위)
* 이 부분을 명료하게 (확실하게) 스케일링 작업

## 모델 배포
* 모델 -> pkl 확장자 -> 배포

In [ ]:
!pip install mlxtend -q

In [ ]:
import joblib # pkl 세이브파일을 만들어주는 라이브러리

joblib.dump(model, 'first_model.pkl')

In [ ]:
from google.colab import files

files.download('first_model.pkl') # 다운로드

In [ ]:
model_from_joblib = joblib.load('first_model.pkl')
model_from_joblib

In [ ]:
pd.Series(model_from_joblib.coef_, index = X.columns)